In [ ]:

# scripts/extract.py (ingest & clean portion)
import pandas as pd
import re
from pathlib import Path

RAW_XLSX = Path("data/raw.xlsx")
ALIASES = {"X", "Drug X", "GenericX", "BrandX"}  # <- expand with your synonyms

def load_and_clean():
    df = pd.read_excel(RAW_XLSX, engine="openpyxl")
    # Normalize column names
    df = df.rename(columns={c: c.strip().lower() for c in df.columns})
    required = ["id", "author_name", "text", "source_type", "date"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    # Filter by any alias of X
    pattern = re.compile(r"\b(" + "|".join(re.escape(a) for a in ALIASES) + r")\b", flags=re.I)
    df = df[df["text"].fillna("").str.contains(pattern)]

    # Basic cleaning
    def clean_text(t):
        t = re.sub(r"http\S+", "", str(t))         # remove URLs
        t = re.sub(r"[@#]\w+", "", t)              # remove @handles and #hashtags (optional)
        t = re.sub(r"\s+", " ", t).strip()
        return t
    df["clean_text"] = df["text"].map(clean_text)

    return df
